In [1]:
import pandas as pd
import parselmouth

from parselmouth.praat import call
import parselmouth

import os

In [2]:


def getFeatures(voiceID, f0min, f0max, unit):
    try:
        sound = parselmouth.Sound(voiceID) # read the sound
        duration = call(sound, "Get total duration") # duration
        pitch = call(sound, "To Pitch", 0.0, f0min, f0max) #create a praat pitch object


        meanF0 = call(pitch, "Get mean", 0, 0, unit) #MDVP:Fo(Hz)
        # get standard deviation
        stdevF0 = call(pitch, "Get standard deviation", 0 ,0, unit) 
        harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 1.0)
        hnr = call(harmonicity, "Get mean", 0, 0) #HNR
        
        pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
        localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:Jitter(%)
        localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:Jitter(Abs)
        rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:RAP
        ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3) #MDVP:PPQ
        ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3) #Jitter:DDP
        localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #MDVP:Shimmer
        localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #MDVP:Shimmer(dB)
        apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #Shimmer:APQ3
        aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6) #Shimmer:APQ5
        apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)

        
        
        return duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer
    except:
        print("file " + voiceID + " was not read because it is not a sound file")
        return -1

In [3]:
def getListOfAudioFiles(dir_path):
    #print list of sound files in directory
    res = []
    # Iterate directory
    for file_path in os.listdir(dir_path):
        # check if current file_path is a file
        if os.path.isfile(os.path.join(dir_path, file_path)):
            # add filename to list
            res.append(file_path)
    return(res)


#print(getListOfAudioFiles('/Users/tomas/Documents/Tese/src/ficheiros'))

In [4]:
pd_audiofiles_path = '../data/PD/'
control_audiofiles_path = '../data/Control'

pd_audiofiles_names = getListOfAudioFiles(pd_audiofiles_path)
control_audiofiles_names = getListOfAudioFiles(control_audiofiles_path)

original_features_df = pd.DataFrame(columns=['Duration', 'meanF0', 'stdevF0', 'hnr', 'localJitter', 'localabsoluteJitter', 'rapJitter', 'ppq5Jitter', 'ddpJitter', 'localShimmer', 'localdbShimmer', 'apq3Shimmer', 'aqpq5Shimmer', 'apq11Shimmer', 'ddaShimmer', 'PD'])


for i in range(len(pd_audiofiles_names)-1):
    values = getFeatures(pd_audiofiles_path+'/'+pd_audiofiles_names[i], 75, 500, "Hertz")
    if values!=-1:
        duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer = values
        original_features_df.loc[len(original_features_df)] = [duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer,1]

for i in range(len(control_audiofiles_names)-1):
    values = getFeatures(control_audiofiles_path+'/'+control_audiofiles_names[i], 75, 500, "Hertz")
    if values!=-1:
        duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer = values
        original_features_df.loc[len(original_features_df)] = [duration, meanF0, stdevF0, hnr, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, ddpJitter, localShimmer, localdbShimmer, apq3Shimmer, aqpq5Shimmer, apq11Shimmer, ddaShimmer,0]




file ../data/PD//.DS_Store was not read because it is not a sound file


In [ ]:
outname = 'original_features.csv'

outdir = '../data/tables/'
if not os.path.exists(outdir):
    os.mkdir(outdir)

original_features_df.to_csv(os.path.join(outdir, outname),index=False)